## CUDA_ERROR_NO_DEVICE Debugging Attempt

Using [this](https://github.com/numba/numba/issues/6777) as reference, checking one by one.

In [1]:
from numba import cuda

In [2]:
print(cuda.gpus)

<Managed Device 0>


In [3]:
!numba -s

System info:
--------------------------------------------------------------------------------
__Time Stamp__
Report started (local time)                   : 2023-09-18 23:55:11.879320
UTC start time                                : 2023-09-19 03:55:11.879323
Running time (s)                              : 5.694455

__Hardware Information__
Machine                                       : x86_64
CPU Name                                      : znver1
CPU Count                                     : 12
Number of accessible CPUs                     : 12
List of accessible CPUs cores                 : 0 1 2 3 4 5 6 7 8 9 10 11
CFS Restrictions (CPUs worth of runtime)      : None

CPU Features                                  : 64bit adx aes avx avx2 bmi bmi2
                                                clflushopt clwb clzero cmov cx16
                                                cx8 f16c fma fsgsbase fxsr lzcnt
                                                mmx movbe pclmul popcnt prfc

In [4]:
import cudf; 
cudf.Series([1,2,3])

ModuleNotFoundError: No module named 'cudf'

"I'm wondering if the CUDA Runtime API (used by cuDF) is initializing something that doesn't get initialized properly by Numba. Can you try running the following and let me know the output please":

In [5]:
from ctypes import CDLL, POINTER, byref, c_size_t
import sys

cudart = CDLL('libcudart.so')

cudaMemGetInfo = cudart.cudaMemGetInfo
cudaMemGetInfo.argtypes = [POINTER(c_size_t), POINTER(c_size_t)]

free = c_size_t()
total = c_size_t()
err = cudaMemGetInfo(byref(free), byref(total))

if (err):
    print(f"cudaMemGetInfo error is {err}")
    sys.exit(1)

print(f"Free: {free.value} bytes")
print(f"Total: {total.value} bytes")

print("Importing Numba")

from numba import cuda

print("Allocating array")

cuda.device_array(1)

print("Finished")

cudaMemGetInfo error is 100


SystemExit: 1

/home/jamesemi/mambaforge/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


I think that adding `export LD_LIBRARY_PATH="/usr/lib/wsl/lib/"` and 
`export NUMBA_CUDA_DRIVER="/usr/lib/wsl/lib/libcuda.so.1"` to the end of `.bashrc` will work.

Yep, that worked, which is why you don't see the error at the top (should've ben patient and ran a copy of the cells down here so as to maintain a record of the error. Anyway!

In [6]:
import torch

In [7]:
torch.zeros([2,4])

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])